<a href="https://colab.research.google.com/github/cardazuluaga/DLCourse/blob/main/Ej62PrediccionSeriesTemporales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
def plot_series(time, series, format="-", start=0, end=None):
    """
    Visualizes time series data

    Args:
      time (array of int) - contains the time steps
      series (array of int) - contains the measurements for each time step
      format - line style when plotting the graph
      label - tag for the line
      start - first time step to plot
      end - last time step to plot
    """

    # Setup dimensions of the graph figure
    plt.figure(figsize=(10, 6))

    if type(series) is tuple:

      for series_num in series:
        # Plot the time series data
        plt.plot(time[start:end], series_num[start:end], format)

    else:
      # Plot the time series data
      plt.plot(time[start:end], series[start:end], format)

    # Label the x-axis
    plt.xlabel("Time")

    # Label the y-axis
    plt.ylabel("Value")

    # Overlay a grid on the graph
    plt.grid(True)

    # Draw the graph on screen
    plt.show()

def trend(time, slope=0):
    """
    Generates synthetic data that follows a straight line given a slope value.

    Args:
      time (array of int) - contains the time steps
      slope (float) - determines the direction and steepness of the line

    Returns:
      series (array of float) - measurements that follow a straight line
    """

    # Compute the linear series given the slope
    series = slope * time

    return series

def seasonal_pattern(season_time):
    """
    Just an arbitrary pattern, you can change it if you wish

    Args:
      season_time (array of float) - contains the measurements per time step

    Returns:
      data_pattern (array of float) -  contains revised measurement values according
                                  to the defined pattern
    """

    # Generate the values using an arbitrary pattern
    data_pattern = np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))

    return data_pattern

def seasonality(time, period, amplitude=1, phase=0):
    """
    Repeats the same pattern at each period

    Args:
      time (array of int) - contains the time steps
      period (int) - number of time steps before the pattern repeats
      amplitude (int) - peak measured value in a period
      phase (int) - number of time steps to shift the measured values

    Returns:
      data_pattern (array of float) - seasonal data scaled by the defined amplitude
    """

    # Define the measured values per period
    season_time = ((time + phase) % period) / period

    # Generates the seasonal data scaled by the defined amplitude
    data_pattern = amplitude * seasonal_pattern(season_time)

    return data_pattern

def noise(time, noise_level=1, seed=None):
    """Generates a normally distributed noisy signal

    Args:
      time (array of int) - contains the time steps
      noise_level (float) - scaling factor for the generated signal
      seed (int) - number generator seed for repeatability

    Returns:
      noise (array of float) - the noisy signal
    """

    # Initialize the random number generator
    rnd = np.random.RandomState(seed)

    # Generate a random number for each time step and scale by the noise level
    noise = rnd.randn(len(time)) * noise_level

    return noise

In [ ]:
# Parameters
time = np.arange(4 * 365 + 1, dtype="float32")
baseline = 10
amplitude = 40
slope = 0.05
noise_level = 5

# Create the series
serie_temporal = baseline + trend(time, slope) + seasonality(time, period=365, amplitude=amplitude)

# Update with noise
serie_temporal += noise(time, noise_level, seed=42)

# Plot the results
plot_series(time, serie_temporal)

In [ ]:
# Definición de la ventana temporal deslizante
ventana_temporal = 20  # Tamaño de la ventana
ventana_salida = 1     # Tamaño de la ventana de salida (predicción a un paso)

# Divide los datos en conjuntos de entrenamiento y prueba
train_split = 1000
datos_entrenamiento = serie_temporal[:train_split]
datos_prueba = serie_temporal[train_split:]
datos_forecasting = serie_temporal[train_split + ventana_temporal:]
t_prueba = time[train_split + ventana_temporal:]

# Genera secuencias y etiquetas utilizando la ventana temporal
generador_entrenamiento = TimeseriesGenerator(
    datos_entrenamiento, datos_entrenamiento, length=ventana_temporal, batch_size=32
)

generador_prueba = TimeseriesGenerator(
    datos_prueba, datos_prueba, length=ventana_temporal, batch_size=1
)

# Construye el modelo MLP
modelo = Sequential([
    Dense(64, activation='relu', input_shape=(ventana_temporal,)),
    Dense(32, activation='relu'),
    Dense(ventana_salida)  # Una sola salida para la predicción
])

modelo.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')

# Entrena el modelo
modelo.fit(generador_entrenamiento, epochs=100)

# Realiza predicciones en el conjunto de prueba
predicciones = modelo.predict(generador_prueba)
predicciones.shape == (predicciones.size,)

fig, ax = plt.subplots()
ax.plot(t_prueba,datos_forecasting,'r')
ax.plot(t_prueba,predicciones,'b')
print('SME:', np.mean(np.square(datos_forecasting - predicciones)))